<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

***Задание 1***

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [114]:
import pandas as pd
import numpy as np

***Решение задания 1***

In [39]:
data = pd.read_csv('ratings.csv', skiprows=0)
"""
Функция возвращает категорию исходя из того, в какой диапазон попадает оценка фильма

"""

def rating_function(rating):
        if 2.0>rating>=4.0:
            return 'средний рейтинг'
        elif rating > 4.0:
            return 'высокий рейтинг'
        return 'средний рейтинг'
# группируем по movieId,а затем вколонке rating считаем среднее,после чего сбрасваем индекс.
data_new = data.groupby('movieId').agg({'rating':'mean'}).reset_index()
# создаем новый столбец class в который вносим результат работы функции rating_function по столбцу rating
data_new['class'] = data_new.rating.apply(rating_function)


data_new.head(10)


,movieId,rating,class
0,1,3.920930,средний рейтинг
1,2,3.431818,средний рейтинг
2,3,3.259615,средний рейтинг
3,4,2.357143,средний рейтинг
4,5,3.071429,средний рейтинг
5,6,3.946078,средний рейтинг
6,7,3.185185,средний рейтинг
7,8,2.875000,средний рейтинг
8,9,3.125000,средний рейтинг
9,10,3.496212,средний рейтинг


***Задание 2***

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

```python
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

```

Результат классификации запишите в отдельный столбец region.



***Решение задания 2***

In [3]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [85]:
cities = pd.read_csv('keywords.csv', skiprows=0)
# создаем функцию, которая иклом проходится по всем значениям колонки keyword и в случае совпадения с элементами в 
# значениях словаря возвращает ключ словаря. Если ничего нет,то возвращается 'undefined'
def geo (keyword): 
    for key, value in geo_data.items():
        for city in value:
            if city in keyword:
                    return key
    return 'undefined'    

cities['region'] = cities.keyword.apply(geo) #применяем функцию к столбцу keyword в data frame cities

cities.query('region != "undefined"') #фильтруем cities по значениям в keyword, неравным undefined (чисто для наглядности результата)

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
...,...,...,...
99590,авито ярославль автомобили с пробегом,3673,Центр
99634,северпост новости мурманской области,3606,Северо-Запад
99808,полармед мурманск запись на прием,3719,Северо-Запад
99890,яндекс метро москва,4548,Центр


***Задание 3 (бонусное)***

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

 - для каждой строки пройдите по всем годам списка years
 - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
 - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
 - Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’
 
 
3. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга



In [116]:
years = [str(i) for i in np.arange(1950, 2021).tolist()] # list comprehention с помощью метода .arange и .tolist из numpy

"""
Функция находит соответствие между годом в списке и годом в названии фильма и возващает его.
Если ничего нет, то возвращает 1900).
"""    

def find_year(title):
    for i in years:
        if i in title:
            return int(i) # возврат в int потребуется для дальнейшего применения метода mean
    return 1900

movies_f = pd.read_csv('movies.csv', skiprows=0) #читаем файл в data frame movies_f
movies_f['year'] = movies_f.title.apply(find_year) #создаем новый столбец и вносим туда значения применяя функцию к title

# объединяем два DF (data_new был создан в первой задаче) по столбцу 'movieId' вариантом левого join
movies = pd.merge(movies_f, data_new, on = 'movieId', how= 'left') 

# для красоты и удобства избавляемся от нескольких ненужных нам столбцов? сначала определив их в columns, а затем 
# удалив их с помощью метода .drop и записав все в новый DF years
columns = ['movieId', 'genres', 'class', 'title']
years = movies.drop(columns, axis=1)

# группируем по году и получаем среднее значение, получаем среднее значение и сортируем по рейтингу по убыванию
years.groupby('year').agg({'rating':'mean'}).sort_values('rating', ascending=False).head(50)

,rating
year,
1971,3.811243
1957,3.767680
1951,3.718044
1952,3.700104
1955,3.672863
1958,3.668078
1976,3.658863
1961,3.656946
1972,3.645005
